# How can I get variables from different tables with 'require_all_on' [#265](https://github.com/intake/intake-esm/issues/265)
conda env: `phd_v3`

In [30]:
!conda list intake

# packages in environment at /home/lalandmi/miniconda3/envs/phd_v3:
#
# Name                    Version                   Build  Channel
intake                    0.6.0                      py_0    conda-forge
intake-esm                2020.6.11                  py_0    conda-forge


In [3]:
import intake

In [4]:
url = "https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json"
col = intake.open_esm_datastore(url)
print(type(col))
col

<class 'intake_esm.core.esm_datastore'>


,unique
activity_id,15
institution_id,33
source_id,76
experiment_id,105
member_id,204
table_id,29
variable_id,373
grid_label,10
zstore,282028
dcpp_init_year,60


In [5]:
col.df.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,AerChemMIP,AS-RCEC,TaiESM1,histSST,r1i1p1f1,AERmon,od550aer,gn,gs://cmip6/AerChemMIP/AS-RCEC/TaiESM1/histSST/...,NaN,20200310
1,AerChemMIP,BCC,BCC-ESM1,histSST,r1i1p1f1,AERmon,mmrbc,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/histSST/r1i...,NaN,20190718
2,AerChemMIP,BCC,BCC-ESM1,histSST,r1i1p1f1,AERmon,mmrdust,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/histSST/r1i...,NaN,20191127
3,AerChemMIP,BCC,BCC-ESM1,histSST,r1i1p1f1,AERmon,mmroa,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/histSST/r1i...,NaN,20190809
4,AerChemMIP,BCC,BCC-ESM1,histSST,r1i1p1f1,AERmon,mmrso4,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/histSST/r1i...,NaN,20191127


In [6]:
query = dict(
    experiment_id=['historical'],
    variable_id=['tas', 'snc', 'pr'],
    table_id=['Amon', 'LImon'],
    source_id=['BCC-CSM2-MR', 'SAM0-UNICON', 'NorESM2-LM']
)

In [26]:
# cat = col.search(require_all_on=["source_id"], **query)
cat = col.search(**query)

In [27]:
cat.df.groupby("source_id")[
    ["experiment_id", "variable_id", "table_id", "member_id"]
].nunique()

,experiment_id,variable_id,table_id,member_id
source_id,,,,
BCC-CSM2-MR,1,3,2,3
NorESM2-LM,1,2,1,3
SAM0-UNICON,1,3,2,1


In [28]:
cat.df.groupby("source_id")["table_id"].unique() 

source_id
BCC-CSM2-MR    [Amon, LImon]
NorESM2-LM            [Amon]
SAM0-UNICON    [Amon, LImon]
Name: table_id, dtype: object

In [29]:
cat.df.groupby("source_id")["variable_id"].unique()    

source_id
BCC-CSM2-MR    [pr, tas, snc]
NorESM2-LM          [pr, tas]
SAM0-UNICON    [pr, tas, snc]
Name: variable_id, dtype: object